# Introduction
We have done feature engineering on the raw dataset and got our final dataset.
Now we will use three different models for predictions:
- Linear Regression
- Decision Tree
- Random Forest Regression
- Regression Enhanced Random Forest

For each model, predictions are made, after a training and parameter tuning phase, on each of the datasets produced in the feat-engineering notebook.
Results for each set of predictions are then plotted and visualized.

## Train Multiple Models

Now that we've tested our data preparation pipeline with a sample model, the next step is to train the data on different regression algorithms to shortlist the most promising algorithms for our problem.

Algorithms to test with include:
- **Linear Regression**: Simple algorithm to implement but can over-simplify real-world problems by assuming a linear relationship among the variables.
- **Adaboost Regressor**: ???
- **Decision Tree**: Powerful model capable of finding complex nonlinear relationships in the data.
- **Random Forest**: Train many Decision Tress on random subsets of the features (*Ensemble Learning*).

# Setup
Let us import the required modules.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import os
from math import sqrt
import matplotlib.pyplot as plt
import pickle

import project.src.feat_eng as fe
import project.src.visualization as viz
import project.src.evaluation as eva

import sklearn.model_selection as modsel
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

%matplotlib inline
sys.path.insert(0, os.path.abspath("../../"))
color = sns.color_palette()
pd.set_option("display.max_columns", 100) #

RNDSTATE = 28

## Load Data
Note that the dataset is already split into Train-Test sets.

In [2]:
engineered_dataset = fe.TrainTestSplit.from_csv_directory(dir_path="../data/lvl4_rfecv")

In [3]:
# not used!!!!!!
full_dataset = pd.concat([engineered_dataset.x_train, engineered_dataset.x_test])

In [4]:
engineered_dataset.x_train.info()
engineered_dataset.y_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62090 entries, 0 to 62089
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   62090 non-null  float64
 1   bedroomcnt                    62090 non-null  float64
 2   fireplacecnt                  62090 non-null  float64
 3   garagecarcnt                  62090 non-null  float64
 4   latitude                      62090 non-null  float64
 5   longitude                     62090 non-null  float64
 6   poolcnt                       62090 non-null  float64
 7   roomcnt                       62090 non-null  float64
 8   threequarterbathnbr           62090 non-null  float64
 9   unitcnt                       62090 non-null  float64
 10  numberofstories               62090 non-null  float64
 11  house_age                     62090 non-null  float64
 12  heatingorsystemtypeid_2.0     62090 non-null  float64
 13  h

array([ 0.01776168,  0.04743217,  0.05062936, ...,  0.03397105,
       -0.08284219,  0.02421837])

promemoria:
-
-
-
- Linear Regression
- Decision Tree
- Random Forest Regression

# Linear Regression Model

-------SCRIVERE QUALCOSA----------------

Linear Regression: Plain linear regression that minimizes the Mean Squared Error(MSE) cost function.

The model RMSE is significantly higher than MAE which suggests that the outliers are affecting the model's performance as RMSE punishes the model more for mispredicting outliers.
The K-Fold Cross Validation shows that the model's performance is highly volatile

In [6]:
linear_reg = LinearRegression()
linear_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

LinearRegression()

In [7]:
linreg_train_pred = linear_reg.predict(engineered_dataset.x_train)
linreg_test_pred = linear_reg.predict(engineered_dataset.x_test)

linreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=linreg_train_pred)
linreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=linreg_test_pred)

In [8]:
print("Training:")
linreg_train_eval.print_eval()
print("Testing:")
linreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07040053572011877

Mean Squared Error : 0.02808442166089148

Root Mean Squared Error : 0.1675840734106063

R2 : 0.004461712937144702

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07225147731871365

Mean Squared Error : 0.033117004168537564

Root Mean Squared Error : 0.1819807796679022

R2 : 0.0015696233476769628



# Decision Tree Regression Model

-------SCRIVERE QUALCOSA----------------

Decision Tree: Powerful model capable of finding complex nonlinear relationships in the data.
Random Forest: Train many Decision Tress on random subsets of the features via the bagging method (Ensemble Learning).

In [ ]:
# just to check if they have the same size
engineered_dataset.x_train.info(verbose=True)
engineered_dataset.y_train.info(verbose=True)

In [14]:
# automatic parameters tuning
dt=DecisionTreeRegressor()
properties={
            'min_samples_leaf':[x for x in range(2,251,2)],
            'max_leaf_nodes':[x for x in range(2,51,2)],
            'max_features': ["sqrt", "log2"]
            }

tuned_dt=GridSearchCV(dt,properties,scoring="neg_mean_squared_error",cv=5,return_train_score=True,verbose=2,n_jobs=-1)
tuned_dt.fit(engineered_dataset.x_train,engineered_dataset.y_train)

print ("Best Score: {:.3f}".format(tuned_dt.best_score_) )
print ("Best Params: ", tuned_dt.best_params_)

Fitting 5 folds for each of 6250 candidates, totalling 31250 fits
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=2; total time=   0.4s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=4; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=6; total time=   0.2s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=8; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=10; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=10; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=12; total time=   0.2s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=14; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=16; total time=   0.3s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=18; total time=   0.2s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=18; total time=

In [15]:
print(tuned_dt.cv_results_)
tuned_dt_results = pd.DataFrame( tuned_dt.cv_results_ )

{'mean_fit_time': array([0.28333716, 0.13964291, 0.16209779, ..., 0.20170031, 0.35152078,
       0.35773363]), 'std_fit_time': array([0.10759404, 0.01030851, 0.05908531, ..., 0.05072802, 0.06148459,
       0.11007492]), 'mean_score_time': array([0.03689189, 0.00696816, 0.03009462, ..., 0.01282024, 0.027981  ,
       0.00799108]), 'std_score_time': array([0.0505381 , 0.00273435, 0.03573632, ..., 0.00878786, 0.02489336,
       0.0042112 ]), 'param_max_features': masked_array(data=['sqrt', 'sqrt', 'sqrt', ..., 'log2', 'log2', 'log2'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_leaf_nodes': masked_array(data=[2, 2, 2, ..., 50, 50, 50],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_leaf': masked_array(data=[2, 4, 6, ..., 246, 248, 250],
             mask=[False, False, False, ..., False, False, False],
       fill_valu

In [16]:
tuned_dt_results[["mean_test_score","mean_train_score"]]

,mean_test_score,mean_train_score
0,-0.028216,-0.028166
1,-0.028216,-0.028186
2,-0.028199,-0.028172
3,-0.028232,-0.028174
4,-0.028202,-0.028175
...,...,...
6245,-0.028230,-0.027935
6246,-0.028217,-0.027932
6247,-0.028242,-0.027953
6248,-0.028249,-0.027935


In [17]:
print(tuned_dt.best_estimator_)
print(tuned_dt.best_score_)

DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=18,
                      min_samples_leaf=4)
-0.02804189485814903


In [18]:
test_acc = mean_squared_error(y_true = engineered_dataset.y_train,
                          y_pred = tuned_dt.predict(engineered_dataset.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

Test Accuracy: 0.02764135515709917


In [19]:
# best parameters from automatic parameters tuning
tree_reg = DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=18, min_samples_leaf=4)
tree_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=18,
                      min_samples_leaf=4)

In [20]:
dtreg_train_pred = tree_reg.predict(engineered_dataset.x_train)
dtreg_test_pred = tree_reg.predict(engineered_dataset.x_test)

dtreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=dtreg_train_pred)
dtreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=dtreg_test_pred)

In [21]:
print("Training:")
dtreg_train_eval.print_eval()
print("Testing:")
dtreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07015647260005224

Mean Squared Error : 0.027537587054007506

Root Mean Squared Error : 0.16594453005148288

R2 : 0.023845939339144917

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07223092120346798

Mean Squared Error : 0.033429312620677214

Root Mean Squared Error : 0.1828368469993869

R2 : -0.00784603043291865



# AdaBoost Regression Model

-------------DA FARE BENE----------- vedere notebook lezione su che parametri ci sono da tunare perche lotto lo fa

In [29]:
# automatic parameters tuning
abreg=AdaBoostRegressor()
properties={
    'base_estimator':[tree_reg],
    'n_estimators':[x for x in range(1,50,2)],
    'learning_rate':[0.001,0.01,0.1,1,10,100]
}

tuned_abreg=GridSearchCV(abreg,properties,scoring="neg_mean_squared_error",cv=5,return_train_score=True,verbose=4,n_jobs=-1)
tuned_abreg.fit(engineered_dataset.x_train,engineered_dataset.y_train)

print ("Best Score: {:.3f}".format(tuned_abreg.best_score_) )
print ("Best Params: ", tuned_abreg.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score: -0.029
Best Params:  {'base_estimator': DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=18,
                      min_samples_leaf=4), 'n_estimators': 1}


In [30]:
print(tuned_abreg.cv_results_)
tuned_abreg_results = pd.DataFrame( tuned_abreg.cv_results_ )

{'mean_fit_time': array([0.26488218, 0.94099331, 1.3051692 , 1.29107695, 2.11149507,
       3.35554562, 3.39018517, 2.11402483, 2.71912079, 3.6848372 ,
       5.06497421, 7.41563287, 4.93092561, 4.92093678, 4.79732475,
       5.06205616, 4.84971786, 3.86176119, 3.656811  , 4.01506152,
       4.20498195, 4.23117948, 4.32121201, 4.59724255, 4.48593793]), 'std_fit_time': array([0.16691373, 0.04588452, 0.32147349, 0.21818145, 0.32903624,
       0.18743325, 0.42595547, 0.2138855 , 0.99797408, 0.70129392,
       1.48895133, 0.67386453, 0.58033487, 0.28612872, 0.23707812,
       0.31313808, 0.74366537, 0.07656587, 0.1620002 , 0.32167362,
       0.21699395, 0.30766411, 0.3161766 , 0.20351934, 0.59455254]), 'mean_score_time': array([0.01521411, 0.04121985, 0.0394238 , 0.04381895, 0.05365973,
       0.12236314, 0.07669239, 0.04939179, 0.11354074, 0.11414528,
       0.21878638, 0.22122545, 0.1382925 , 0.13757067, 0.15553842,
       0.145509  , 0.16178989, 0.12024722, 0.12148223, 0.11539307,
     

In [31]:
tuned_abreg_results[["param_n_estimators","mean_test_score","mean_train_score"]]

,param_n_estimators,mean_test_score,mean_train_score
0,1,-0.028536,-0.027919
1,3,-0.028796,-0.028210
2,5,-0.028900,-0.028295
3,7,-0.032165,-0.030976
4,9,-0.036423,-0.035001
5,11,-0.038258,-0.036502
6,13,-0.047705,-0.045464
7,15,-0.051427,-0.048791
8,17,-0.060699,-0.058274
9,19,-0.072658,-0.069087


In [32]:
print(tuned_abreg.best_estimator_)
print(tuned_abreg.best_score_)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_features='sqrt',
                                                       max_leaf_nodes=18,
                                                       min_samples_leaf=4),
                  n_estimators=1)
-0.02853591844479152


In [33]:
test_acc = mean_squared_error(y_true = engineered_dataset.y_train,
                          y_pred = tuned_abreg.predict(engineered_dataset.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

Test Accuracy: 0.02775105903140737


In [35]:
# best parameters from automatic parameters tuning
adaboost_reg = AdaBoostRegressor(n_estimators=1,learning_rate=1)
adaboost_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

AdaBoostRegressor(n_estimators=1)

In [36]:
adaboostreg_train_pred = adaboost_reg.predict(engineered_dataset.x_train)
adaboostreg_test_pred = adaboost_reg.predict(engineered_dataset.x_test)

adaboostreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=adaboostreg_train_pred)
adaboostreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=adaboostreg_test_pred)

In [37]:
print("Training:")
adaboostreg_train_eval.print_eval()
print("Testing:")
adaboostreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07021495210436013

Mean Squared Error : 0.027993827617312905

Root Mean Squared Error : 0.16731356076933185

R2 : 0.00767309610363176

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07222335548425936

Mean Squared Error : 0.033415456214150346

Root Mean Squared Error : 0.1827989502545087

R2 : -0.0074282795664073475



# Random Forest Regression Model

-------SCRIVERE QUALCOSA----------------

fare GridSearchCV per il tuning di parametri cosi ho un motivo per cui sto usando certi parametri

uso parametri trovati per decision tree

In [ ]:
RF_HYPER_PARAMS = { # riempire con risultati del gridsearchsv
    "n_estimators": [150, 200, 250],
    "min_samples_leaf": [1, 50, 100, 200],
    "max_leaf_nodes": [2, 5, 10],
    "max_features": ["sqrt", "log2"]
}

In [53]:
forest_reg = RandomForestRegressor()
search_result = tr.grid_search_cv_tuning(
                                         train_data=data.x_train.values,
                                         train_target=data.y_train,
                                         scoring="neg_mean_squared_error",)

tuned_model = modsel.HalvingGridSearchCV(forest_reg, RF_HYPER_PARAMS, cv=5, verbose=2,
                                         n_jobs=-1, **gridsearchcv_kwargs)
tuned_model.fit(train_data, train_target)

search_result = tuned_model

print("Tuning results:")
print(f"Best params: {search_result.best_params_}")

print("Fitting with best params and full training set...")
tuned_rf = RandomForestRegressor(n_jobs=N_JOBS, random_state=RND_SEED,
                                 **search_result.best_params_)
tuned_rf.fit(X=data.x_train.values, y=data.y_train)



forest_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

RandomForestRegressor(max_depth=6, n_estimators=50)

In [ ]:
rf=RandomForestRegressor(n_estimators=50,oob_score=False) ??
# automatic parameters tuning
rfreg=RandomForestRegressor()
properties={
    #min_sample_leaf piu piccolo possibile
    #max feature piu grande possibile
    #min_samples_split piu' piccolo possibile
    #max_leaf_nodes piu' grande possibile
    #max_samples piu' alto possibile
}

tuned_rfreg=GridSearchCV(rfreg,properties,scoring="neg_mean_squared_error",cv=5,return_train_score=True,verbose=4,n_jobs=-1)
tuned_rfreg.fit(engineered_dataset.x_train,engineered_dataset.y_train)

print ("Best Score: {:.3f}".format(tuned_rfreg.best_score_) )
print ("Best Params: ", tuned_rfreg.best_params_)

In [ ]:
print(tuned_rfreg.cv_results_)
tuned_rfreg_results = pd.DataFrame( tuned_rfreg.cv_results_ )

In [ ]:
tuned_rfreg_results[["param_n_estimators","mean_test_score","mean_train_score"]]

In [ ]:
print(tuned_rfreg.best_estimator_)
print(tuned_rfreg.best_score_)

In [ ]:
test_acc = mean_squared_error(y_true = engineered_dataset.y_train,
                          y_pred = tuned_rfreg.predict(engineered_dataset.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

In [84]:
# best parameters from automatic parameters tuning
forest_reg = RandomForestRegressor(n_estimators=2,min_samples_leaf=2,max_leaf_nodes=2,max_features="sqrt")
forest_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

In [ ]:
rfreg_train_pred = forest_reg.predict(engineered_dataset.x_train)
rfreg_test_pred = forest_reg.predict(engineered_dataset.x_test)

rfreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=rfreg_train_pred)
rfreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=rfreg_test_pred)

In [86]:
print("Training:")
rfreg_train_eval.print_eval()
print("Testing:")
rfreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.0696034486709443

Mean Squared Error : 0.026873474451725804

Root Mean Squared Error : 0.1639313101629027

R2 : 0.047387443254572004

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07203089965868967

Mean Squared Error : 0.03335425261038464

Root Mean Squared Error : 0.18263146664905433

R2 : -0.005583078326304669



# Performance Visualization

### Setup

## Training Performance

## Testing Performance

## Training vs Testing

## Predictions and Residuals

### Linear Regression

### Decision Forest Regression

### AdaBoost Regression

### Random Forest Regression

### A Look at the Expected Value and Variance of True log-errors

## Best and Worst Instances

### Linear Regression

### Decision Forest Regression

### AdaBoost Regression

### Random Forest Regression

----------------------- DA NOTEBOOK CRASTO -----------------------

## Model Evaluation

### Baseline Metrics

It is important to set a baseline for the model's performance to compare different algorithms. For regression problems, the baseline metrics are calculated by replacing $y'$ with $\bar{y}$. Using this, the different baseline regression metrics are:

- **MSE Baseline**: Variance of the target variable (Mean Squared Error)
- **RMSE Baseline**: Standard Deviation of the target variable (Root Mean Squared Error)
- **MAE Baseline**: Average Abolsute Deviation of the target variable (Mean Absolute Error)
- **R2 Baseline**: 0

For this regression problem, we will use the models' **Mean Absolute Error** and **RMSE (Root Mean Squared Error)** to compare the different algorithms which have **baseline values of 0.533 and 0.0837** respectively.

We will also observe the RMSE as another evaluation metric which punishes more for outliers than MAE.

In [ ]:
# Baseline for RMSE
print(f"MAE Baseline: {engineered_dataset.y_train.mad()}")
print(f"RMSE Baseline: {engineered_dataset.y_train.std()}")

### MAE Evaluation

To evaluate and short list the most promising models, we will use the models' **MAE** in two different ways:

1) **MAE on Validation Set**: Calculates the MAE on the validation set which is quicker to calculate than evaluation using Cross-Validation. However, it is possible the MAE obtained is skewed depending on the instances sampled in the validation set.

2) A great alternative is to use **K-Fold Cross-Validation** where the training set is randomly split into `n` subsets (for example 10 subsets) called *folds*. It trains and evaluates the model 10 times, picking a different fold for evaluation every time and training on the other 9 folds. Result is an array containing the 10 evaluation scores. Takes longer to evaluate but provides a more accurate measure of the model's performance.

In [23]:
def get_eval_metrics(models, X, y_true):
    """
    Calculates MAE (Mean Absoulate Error) and RMSE (Root Mean Squared Error) on the data set for input models.
    `models`: list of fit models
    """
    for model in models:
        y_pred= model.predict(X)
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        mae = mean_absolute_error(y_true, y_pred)
        print(f"Model: {model}")
        print(f"MAE: {mae}, RMSE: {rmse}")

# Test usage of RMSE function
# get_eval_metrics([lin_reg, ridge_reg, lasso_reg], X_prepared_val, y_val)

In [24]:
def display_scores(model, scores):
    print("-"*50)
    print("Model:", model)
    print("\nScores:", scores)
    print("\nMean:", scores.mean())
    print("\nStandard deviation:", scores.std())

def get_cross_val_scores(models, X, y, cv=10, fit_params=None):
    """
    Performs k-fold cross validation and calculates MAE for each fold for all input models.
    `models`: list of fit models
    """
    for model in models:
        mae = -cross_val_score(model, X, y, scoring="neg_mean_absolute_error", cv=cv, fit_params=fit_params)
        display_scores(model, mae)

    # Test usage of cross val function
# get_cross_val_scores([lin_reg, ridge_reg], X_prepared, y_train, cv=5)

# Random Forest Regression Model¶

In [12]:
forest_reg = RandomForestRegressor(n_estimators= 50, max_depth=6)

forest_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

/var/folders/_c/vnjyd7f9581cdkdmh1wpts080000gn/T/ipykernel_77011/3605848252.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)


RandomForestRegressor(max_depth=6, n_estimators=50)

In [13]:
forest_reg_pred = forest_reg.predict(engineered_dataset.x_test)

print('Mean Absolute Error : {}'.format(mean_absolute_error(engineered_dataset.y_test, forest_reg_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(engineered_dataset.y_test, forest_reg_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(engineered_dataset.y_test, forest_reg_pred))))

Mean Absolute Error : 0.07201648566472933

Mean Squared Error : 0.03331842958157641

Root Mean Squared Error : 0.18253336566659917


# Cross Validation & Hyperparameter Optimization for Random Forest

In [ ]:
scores = cross_val_score(forest_reg, engineered_dataset.x_train, engineered_dataset.y_train, scoring="neg_mean_squared_error", cv = 5)

In [17]:
forest_reg_rmse_scores = np.sqrt(-scores)
forest_reg_rmse_scores

array([0.16437461, 0.16956266, 0.16824223, 0.15338318, 0.17947782])

In [21]:
param_grid = [
    {'n_estimators': [3, 4, 5], 'max_features': [2, 4, 6]},
    {'bootstrap': [False], 'n_estimators': [3, 6, 9], 'max_features': [2, 4, 6]}]

forest_regressor = RandomForestRegressor()

grid_search = GridSearchCV(forest_regressor, param_grid, scoring='neg_mean_squared_error',return_train_score=True,cv=3)

In [ ]:
# grid_search.fit(engineered_dataset.x_train, engineered_dataset.y_train)

In [25]:
grid_search.best_params_

{'max_features': 4, 'n_estimators': 500}

In [26]:
grid_search.best_estimator_

RandomForestRegressor(max_features=4, n_estimators=500)

In [ ]:
# final_predictor = grid_search.best_estimator_
# final_predictor.fit(engineered_dataset.x_train, engineered_dataset.y_train)
# final_pred = final_predictor.predict(engineered_dataset.x_test)

In [ ]:
# print('Mean Absolute Error : {}'.format(mean_absolute_error(engineered_dataset.y_test, final_pred)))
# print()
# print('Mean Squared Error : {}'.format(mean_squared_error(engineered_dataset.y_test, final_pred)))
# print()
# print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(engineered_dataset.y_test, final_pred))))

In [ ]:
# saving the model
# file_name = 'final_pickle_model.pickle'
# pickle.dump(final_predictor,open(file_name,'wb'))

# Feature importance

In [ ]:
# feature_importances = grid_search.best_estimator_.feature_importances_
#
# attrs = list(engineered_dataset.select_dtypes(include = ['float64','int64']))
#
# sorted(zip(attrs, feature_importances), reverse=True)

# Saving Predictions

In [ ]:
# model_pred = pd.DataFrame({'parcelid':X_test_new.parcelid, 'logerror':final_pred})
# model_pred.to_csv('model_predictions.csv',index=False)
# model_pred.head()

# Conclusion

1. I have performed all the feature engineering steps necessary to ensure the dataset is ready to be fed into Machine Learning algorithms.

2. After Pre-processing and Feature Engineering the raw dataset we splitted the dataset into train and test sets.

3. Performed Feature scaling on data for better performance.

4. Trained multiple models using different ML regression algorithms on dataset.

5. Appleied Performance metrics such as MAE, MSE, RMSE to find out best prediction model.

6. With the help of GridSearch CV we found out best estimator with least Root mean squred error.

7. Saved best predictor in .pickle format for future predictions.

8. Done prediction on test data and saved predictions into .csv file.